In [3]:
import requests
from bs4 import BeautifulSoup as bs
import pymongo
import re

In [2]:
conn = pymongo.MongoClient()
cine_db = conn.cine21
actors = cine_db.actors

In [3]:
cine21_url = "http://www.cine21.com/rank/person/content"
header = {}
header['section'] = 'actor'
header['period_start'] = '2020-03'
header['gender'] = 'all'
header['page'] = 1

res = requests.post(cine21_url, data=header)

In [4]:
soup = bs(res.content, 'html.parser')
print(soup)

 <ul class="people_list">
<li class="people_li">
<a href="/db/person/info/?person_id=2552"><img alt="" class="people_thumb" src="http://image.cine21.com/resize/cine21/person/2019/1211/11_32_33__5df055413c01a[X145,145].jpg" target="_blank"/></a>
<div class="name"><a href="/db/person/info/?person_id=2552">전도연(1편)</a></div>
<ul class="num_info">
<li><span class="tit">흥행지수</span><strong>71,991</strong></li>
<!--
						<li><a href="#" class="btn_graph"><span class="ico"></span><span>흥행성적<br />그래프로 보기</span></a></li>
						-->
</ul>
<!-- 영화포스터는 최대 5개까지만 -->
<ul class="mov_list">
<li>
<a href="/movie/info/?movie_id=54143">
<img alt="" class="thumb" src="http://image.cine21.com/resize/cine21/poster/2020/0212/16_42_16__5e43ac5844ca0[X85,120].jpg" target="_blank"/>
<span>지푸라기라도 잡고 싶은 짐승들</span>
</a>
</li>
</ul>
<!-- 순위 --><span class="grade">1</span>
</li>
<li class="people_li">
<a href="/db/person/info/?person_id=13093"><img alt="" class="people_thumb" src="http://image.cine21.com/resize/cine21

In [5]:
actors = soup.select("ul.people_list li.people_li div.name")

actor_list = []
for actor in actors:
    actor_name = re.sub('\(\w*\)','',actor.text)
    actor_link = actor.select_one('a').attrs['href']
    actor_links = requests.get("http://www.cine21.com/"+actor_link)
    actor_response = bs(actor_links.content, 'html.parser')
    default_info = actor_response.select_one("ul.default_info")
    actor_details = default_info.select("li")
    
    actor_dict = {}
    for actor_detail in actor_details:
        actor_schema = actor_detail.select_one('span.tit').text
        actor_value = re.sub("<span.*?>.*?</span>",'',str(actor_detail))
        actor_value = re.sub("<.*?>",'',actor_value).strip()
        actor_dict[actor_schema] = actor_value
    actor_list.append(actor_dict)
print(actor_list)

[{'다른 이름': 'Jeon Do Youn', '직업': '배우', '생년월일': '1973-02-11', '성별': '여', '신장/체중': '165cm, 45kg', '학교': '서울예술대학 방송연예', '취미': '음악듣기, 영화감상', '특기': '수상스키, 포켓볼', '소속사': '노아엔터테인먼트'}, {'원어명': '윤여정', '직업': '배우', '생년월일': '1947-06-19', '성별': '여', '학교': '한양대학교 국문학 (중퇴)'}, {'직업': '배우', '생년월일': '1973-03-20', '성별': '남', '홈페이지': 'https://www.instagram.com/tojws/\nhttps://www.facebook.com/officialjws', '신장/체중': '186cm, 79kg', '학교': '경기상업고등학교 중퇴', '취미': '비디오 감상，드라이브', '특기': '농구，수영', '소속사': '(주)토러스 필름'}, {'직업': '배우', '생년월일': '1972-11-21', '성별': '남', '신장/체중': '177cm, 72kg'}, {'직업': '배우', '생년월일': '1975-03-06', '성별': '여', '홈페이지': 'https://www.facebook.com/cjesmiran'}, {'직업': '배우', '생년월일': '1982-05-22', '성별': '남', '홈페이지': 'https://twitter.com/k_moo'}, {'직업': '배우', '성별': '남'}]


In [35]:
# 배우 이름과 흥행지수 따오기
actors_info_list = []

actors = soup.select("ul.people_list li.people_li")
hits = soup.select("ul.people_list li.people_li ul.num_info li strong")
movies = soup.select("ul.mov_list")
for index, actor in enumerate(actors):
    actor_name = actor.select_one("div.name")
    actor_name = re.sub("\(\w*\)",'',actor_name.text)
    actor_hits = int(hits[index].text.replace(',',''))
    movie_titles = movies[index].select('li a span')
    movie_titles_list = []
    for movie_title in movie_titles:
        movie_titles_list.append(movie_title.text)
    
    actor_info_dict = {}
    actor_info_dict['배우이름'] = actor_name
    actor_info_dict['흥행지수'] = actor_hits
    actor_info_dict['출연작'] = movie_titles_list
    
    actors_link = actor.select_one('a').attrs['href']
    actor_links = requests.get("http://www.cine21.com/"+actors_link)
    actor_response = bs(actor_links.content, 'html.parser')
    default_info = actor_response.select_one("ul.default_info")
    actor_details = default_info.select("li")
    for actor_detail in actor_details:
        actor_schema = actor_detail.select_one('span.tit').text
        actor_value = re.sub("<span.*?>.*?</span>",'',str(actor_detail))
        actor_value = re.sub("<.*?>",'',actor_value).strip()
        actor_info_dict[actor_schema] = actor_value
    actors_info_list.append(actor_info_dict)
print(actors_info_list)
    

[{'배우이름': '전도연', '흥행지수': 71991, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '다른 이름': 'Jeon Do Youn', '직업': '배우', '생년월일': '1973-02-11', '성별': '여', '신장/체중': '165cm, 45kg', '학교': '서울예술대학 방송연예', '취미': '음악듣기, 영화감상', '특기': '수상스키, 포켓볼', '소속사': '노아엔터테인먼트'}, {'배우이름': '윤여정', '흥행지수': 69492, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '찬실이는 복도 많지'], '원어명': '윤여정', '직업': '배우', '생년월일': '1947-06-19', '성별': '여', '학교': '한양대학교 국문학 (중퇴)'}, {'배우이름': '정우성', '흥행지수': 65448, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '직업': '배우', '생년월일': '1973-03-20', '성별': '남', '홈페이지': 'https://www.instagram.com/tojws/\nhttps://www.facebook.com/officialjws', '신장/체중': '186cm, 79kg', '학교': '경기상업고등학교 중퇴', '취미': '비디오 감상，드라이브', '특기': '농구，수영', '소속사': '(주)토러스 필름'}, {'배우이름': '배성우', '흥행지수': 58900, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '직업': '배우', '생년월일': '1972-11-21', '성별': '남', '신장/체중': '177cm, 72kg'}, {'배우이름': '라미란', '흥행지수': 51681, '출연작': ['정직한 후보'], '직업': '배우', '생년월일': '1975-03-06', '성별': '여', '홈페이지': 'https://www.facebook.com/cjesmiran'}, {'배우이름': '김무열', '흥행지수': 47375, '출연작': ['정직한 후보

## 최종 단계
- pymongo 연결
- 필요한 라이브러리 import
- 크롤링코드 작성
- collection.insert_many로 MongoDB에 데이터 입력

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pymongo

conn = pymongo.MongoClient()
cine21 = conn.cine21
actors_db = cine21.actors_db

actors_info_list = []

cine21_url = "http://www.cine21.com/rank/person/content"
header = {}
header['section'] = 'actor'
header['period_start'] = '2020-03'
header['gender'] = 'all'
for index in range(1,21):
    header['page'] = index

    res = requests.post(cine21_url, data=header)
    soup = bs(res.content, 'html.parser')
    actors = soup.select("ul.people_list li.people_li")
    hits = soup.select("ul.people_list li.people_li ul.num_info li strong")
    movies = soup.select("ul.mov_list")
    rankings = soup.select('ul.people_list li.people_li')
    for index, actor in enumerate(actors):
        actor_info_dict = {}
        actor_ranking = rankings[index].select_one('span.grade').text
        actor_name = actor.select_one("div.name")
        actor_name = re.sub("\(\w*\)",'',actor_name.text)
        actor_hits = int(hits[index].text.replace(',',''))
        movie_titles = movies[index].select('li a span')
        movie_titles_list = []
        for movie_title in movie_titles:
            movie_titles_list.append(movie_title.text)
        
        actor_info_dict['배우순위'] = actor_ranking
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연작'] = movie_titles_list

        actors_link = actor.select_one('a').attrs['href']
        actor_links = requests.get("http://www.cine21.com/"+actors_link)
        actor_response = bs(actor_links.content, 'html.parser')
        default_info = actor_response.select_one("ul.default_info")
        actor_details = default_info.select("li")
        for actor_detail in actor_details:
            actor_schema = actor_detail.select_one('span.tit').text
            actor_value = re.sub("<span.*?>.*?</span>",'',str(actor_detail))
            actor_value = re.sub("<.*?>",'',actor_value).strip()
            actor_info_dict[actor_schema] = actor_value
        actors_info_list.append(actor_info_dict)

In [4]:
conn = pymongo.MongoClient()
cine21 = conn.cine21
actors_db = cine21.actors_db

In [15]:
actors_db.update_many({}, {'$rename': {'배우이름': '배우 이름'}})

In [5]:
docs = actors_db.find({}).limit(5)
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8869c8155a38268c005b64'), '배우순위': '1', '흥행지수': 70877, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '다른 이름': 'Jeon Do Youn', '직업': '배우', '생년월일': '1973-02-11', '성별': '여', '신장/체중': '165cm, 45kg', '학교': '서울예술대학 방송연예', '취미': '음악듣기, 영화감상', '특기': '수상스키, 포켓볼', '소속사': '노아엔터테인먼트', '배우 이름': '전도연'}
{'_id': ObjectId('5e8869c8155a38268c005b65'), '배우순위': '2', '흥행지수': 68603, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '찬실이는 복도 많지'], '원어명': '윤여정', '직업': '배우', '생년월일': '1947-06-19', '성별': '여', '학교': '한양대학교 국문학 (중퇴)', '배우 이름': '윤여정'}
{'_id': ObjectId('5e8869c8155a38268c005b66'), '배우순위': '3', '흥행지수': 64632, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '비트'], '직업': '배우', '생년월일': '1973-03-20', '성별': '남', '홈페이지': 'https://www.instagram.com/tojws/\nhttps://www.facebook.com/officialjws', '신장/체중': '186cm, 79kg', '학교': '경기상업고등학교 중퇴', '취미': '비디오 감상，드라이브', '특기': '농구，수영', '소속사': '(주)토러스 필름', '배우 이름': '정우성'}
{'_id': ObjectId('5e8869c8155a38268c005b67'), '배우순위': '4', '흥행지수': 57989, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '직업': '배우', '생년월일': '1972-11-21', '성

In [20]:
                        # sort에서 디폴트값은 sort한 속성이 없는 순으로 출력됨!!!!
docs = actors_db.find({}).sort('배우순위',pymongo.DESCENDING).limit(5)
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8869c8155a38268c005bc6'), '배우순위': '99', '흥행지수': 629, '출연작': ['아내를 죽였다'], '다른 이름': '이보연', '직업': '배우', '생년월일': '1982-07-03', '성별': '남', '홈페이지': 'https://www.instagram.com/lee_si_eon/\nhttps://twitter.com/dm33482', '배우 이름': '이시언'}
{'_id': ObjectId('5e8869c8155a38268c005bc5'), '배우순위': '98', '흥행지수': 661, '출연작': ['눈의 여왕4'], '직업': '성우', '성별': '남', '배우 이름': '임채헌'}
{'_id': ObjectId('5e8869c8155a38268c005bc4'), '배우순위': '97', '흥행지수': 665, '출연작': ['리틀 포레스트'], '직업': '배우', '생년월일': '1986-09-25', '성별': '남', '홈페이지': 'https://www.instagram.com/ryusdb/', '배우 이름': '류준열'}
{'_id': ObjectId('5e8869c8155a38268c005bc3'), '배우순위': '96', '흥행지수': 671, '출연작': ['살인의 추억'], '다른 이름': '宋在浩;송재언', '직업': '배우', '생년월일': '1939-03-10', '성별': '남', '신장/체중': '165cm, 65kg', '학교': '동아대학교 국어국문학', '취미': '촬영', '특기': '사격', '배우 이름': '송재호'}
{'_id': ObjectId('5e8869c8155a38268c005bc2'), '배우순위': '95', '흥행지수': 674, '출연작': ['이장'], '직업': '배우', '성별': '남', '배우 이름': '강민준'}


In [21]:
docs = actors_db.find({'특기':{'$exists':True}}).sort('배우순위', pymongo.DESCENDING).limit(5)
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8869c8155a38268c005bc3'), '배우순위': '96', '흥행지수': 671, '출연작': ['살인의 추억'], '다른 이름': '宋在浩;송재언', '직업': '배우', '생년월일': '1939-03-10', '성별': '남', '신장/체중': '165cm, 65kg', '학교': '동아대학교 국어국문학', '취미': '촬영', '특기': '사격', '배우 이름': '송재호'}
{'_id': ObjectId('5e8869c8155a38268c005bbf'), '배우순위': '92', '흥행지수': 732, '출연작': ['백두산', '남산의 부장들'], '다른 이름': 'Byung-hun Lee;BH Lee', '직업': '배우', '생년월일': '1970-07-12', '성별': '남', '홈페이지': 'http://www.leebyunghun.kr/', '신장/체중': '177cm, 72kg', '학교': '한양대학교 불어문학과', '취미': '모자수집, 여행', '특기': '태권도, 스노우보드, 수영, 팔씨름', '배우 이름': '이병헌'}
{'_id': ObjectId('5e8869c8155a38268c005bb1'), '배우순위': '78', '흥행지수': 945, '출연작': ['불한당: 나쁜 놈들의 세상'], '직업': '배우', '생년월일': '1978-09-15', '성별': '남', '신장/체중': '180cm, 70kg', '취미': '낚시, 당구', '특기': '스노우보드', '배우 이름': '김성오'}
{'_id': ObjectId('5e8869c8155a38268c005ba2'), '배우순위': '63', '흥행지수': 1261, '출연작': ['살인의 추억'], '직업': '배우', '생년월일': '1972-06-01', '성별': '남', '신장/체중': '183cm, 78kg', '학교': '중앙대학교 연극학과', '취미': '낚시, 등산, 독서(고전 읽기), 스포츠', '특기'

In [32]:
docs = actors_db.find({'생년월일':{'$exists':True}}, {'_id':0,'배우 이름':1})
for doc in docs:
    print(doc)

{'배우 이름': '전도연'}
{'배우 이름': '윤여정'}
{'배우 이름': '정우성'}
{'배우 이름': '배성우'}
{'배우 이름': '라미란'}
{'배우 이름': '김무열'}
{'배우 이름': '나문희'}
{'배우 이름': '신현빈'}
{'배우 이름': '윤경호'}
{'배우 이름': '정가람'}
{'배우 이름': '송영창'}
{'배우 이름': '진경'}
{'배우 이름': '온주완'}
{'배우 이름': '성훈'}
{'배우 이름': '조한철'}
{'배우 이름': '강말금'}
{'배우 이름': '박지환'}
{'배우 이름': '손종학'}
{'배우 이름': '김준한'}
{'배우 이름': '김영민'}
{'배우 이름': '조수향'}
{'배우 이름': '김소은'}
{'배우 이름': '윤승아'}
{'배우 이름': '윤세아'}
{'배우 이름': '허동원'}
{'배우 이름': '지창욱'}
{'배우 이름': '송강호'}
{'배우 이름': '배유람'}
{'배우 이름': '김소현'}
{'배우 이름': '김용림'}
{'배우 이름': '이레'}
{'배우 이름': '하정우'}
{'배우 이름': '심은경'}
{'배우 이름': '장혜진'}
{'배우 이름': '김남길'}
{'배우 이름': '박소담'}
{'배우 이름': '이정은'}
{'배우 이름': '오지호'}
{'배우 이름': '이선균'}
{'배우 이름': '허율'}
{'배우 이름': '김상호'}
{'배우 이름': '조여정'}
{'배우 이름': '김시아'}
{'배우 이름': '양정화'}
{'배우 이름': '정지소'}
{'배우 이름': '한동규'}
{'배우 이름': '전혜진'}
{'배우 이름': '서영화'}
{'배우 이름': '정현준'}
{'배우 이름': '설경구'}
{'배우 이름': '임시완'}
{'배우 이름': '김상경'}
{'배우 이름': '김희원'}
{'배우 이름': '이경영'}
{'배우 이름': '공민정'}
{'배우 이름': '김뢰하'}
{'배우 이름': '신린아'}
{'배우 이름': '고서희'}
{'배우 이름': '박서준'}
{

In [6]:
actors_db.update_many({}, {'$rename':{'배우 이름':'배우이름'}})

In [10]:
docs = actors_db.find({}, {'배우이름':1,'생년월일':1, '_id':0}).sort('생년월일', pymongo.DESCENDING).limit(5)
for doc in docs:
    print(doc)

{'생년월일': '2011-11-08', '배우이름': '정현준'}
{'생년월일': '2009-06-26', '배우이름': '허율'}
{'생년월일': '2009-05-14', '배우이름': '신린아'}
{'생년월일': '2008-05-06', '배우이름': '김시아'}
{'생년월일': '2006-03-12', '배우이름': '이레'}


In [11]:
docs = actors_db.find({'특기':{'$exists':True}}).sort('순위').limit(5)
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8869c8155a38268c005b64'), '배우순위': '1', '흥행지수': 70877, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '다른 이름': 'Jeon Do Youn', '직업': '배우', '생년월일': '1973-02-11', '성별': '여', '신장/체중': '165cm, 45kg', '학교': '서울예술대학 방송연예', '취미': '음악듣기, 영화감상', '특기': '수상스키, 포켓볼', '소속사': '노아엔터테인먼트', '배우이름': '전도연'}
{'_id': ObjectId('5e8869c8155a38268c005b66'), '배우순위': '3', '흥행지수': 64632, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '비트'], '직업': '배우', '생년월일': '1973-03-20', '성별': '남', '홈페이지': 'https://www.instagram.com/tojws/\nhttps://www.facebook.com/officialjws', '신장/체중': '186cm, 79kg', '학교': '경기상업고등학교 중퇴', '취미': '비디오 감상，드라이브', '특기': '농구，수영', '소속사': '(주)토러스 필름', '배우이름': '정우성'}
{'_id': ObjectId('5e8869c8155a38268c005b7f'), '배우순위': '28', '흥행지수': 4788, '출연작': ['너의 이름은.'], '직업': '배우', '생년월일': '1987-07-05', '성별': '남', '홈페이지': 'https://twitter.com/jichangwook\nhttps://www.instagram.com/jichangwook/', '신장/체중': '180cm, 65kg', '학교': '단국대학교 공연영상학부', '특기': '노래, 유도, 수영', '소속사': '(주)글로리어스', '배우이름': '지창욱'}
{'_id': ObjectId('5e8869c8155a38268

In [16]:
docs = actors_db.find({'흥행지수':{'$gte':100}, '출연작':'지푸라기라도 잡고 싶은 짐승들'}, {'배우이름':1,'흥행지수':1,'_id':0})
for doc in docs:
    print(doc)

{'흥행지수': 70877, '배우이름': '전도연'}
{'흥행지수': 68603, '배우이름': '윤여정'}
{'흥행지수': 64632, '배우이름': '정우성'}
{'흥행지수': 57989, '배우이름': '배성우'}
{'흥행지수': 45207, '배우이름': '정만식'}
{'흥행지수': 40142, '배우이름': '신현빈'}
{'흥행지수': 32216, '배우이름': '정가람'}
{'흥행지수': 25773, '배우이름': '진경'}
{'흥행지수': 19331, '배우이름': '박지환'}
{'흥행지수': 12888, '배우이름': '김준한'}
{'흥행지수': 6549, '배우이름': '허동원'}


In [22]:
docs = actors_db.find({'$or' : [{'출연작':'지푸라기라도 잡고 싶은 짐승들'},{'출연작':'극한직업'}]}, { '_id':0})
for doc in docs:
    print(doc)

{'배우순위': '1', '흥행지수': 70877, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '다른 이름': 'Jeon Do Youn', '직업': '배우', '생년월일': '1973-02-11', '성별': '여', '신장/체중': '165cm, 45kg', '학교': '서울예술대학 방송연예', '취미': '음악듣기, 영화감상', '특기': '수상스키, 포켓볼', '소속사': '노아엔터테인먼트', '배우이름': '전도연'}
{'배우순위': '2', '흥행지수': 68603, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '찬실이는 복도 많지'], '원어명': '윤여정', '직업': '배우', '생년월일': '1947-06-19', '성별': '여', '학교': '한양대학교 국문학 (중퇴)', '배우이름': '윤여정'}
{'배우순위': '3', '흥행지수': 64632, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '비트'], '직업': '배우', '생년월일': '1973-03-20', '성별': '남', '홈페이지': 'https://www.instagram.com/tojws/\nhttps://www.facebook.com/officialjws', '신장/체중': '186cm, 79kg', '학교': '경기상업고등학교 중퇴', '취미': '비디오 감상，드라이브', '특기': '농구，수영', '소속사': '(주)토러스 필름', '배우이름': '정우성'}
{'배우순위': '4', '흥행지수': 57989, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '직업': '배우', '생년월일': '1972-11-21', '성별': '남', '신장/체중': '177cm, 72kg', '배우이름': '배성우'}
{'배우순위': '7', '흥행지수': 45207, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '부당거래', '창궐'], '직업': '배우', '성별': '남', '배우이름': '정만식'}
{'배우순위': '9', '흥행지수': 401

In [23]:
docs = actors_db.find({'흥행지수':{'$gte':100}, '$or':[{'출연작':'지푸라기라도 잡고 싶은 짐승들'},{'출연작':'극한직업'}]},
                     {'흥행지수':1, '배우이름':1, '출연작':1, '_id':0})
for doc in docs:
    print(doc)

{'흥행지수': 70877, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '배우이름': '전도연'}
{'흥행지수': 68603, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '찬실이는 복도 많지'], '배우이름': '윤여정'}
{'흥행지수': 64632, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '비트'], '배우이름': '정우성'}
{'흥행지수': 57989, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '배우이름': '배성우'}
{'흥행지수': 45207, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '부당거래', '창궐'], '배우이름': '정만식'}
{'흥행지수': 40142, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '클로젯'], '배우이름': '신현빈'}
{'흥행지수': 32216, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '배우이름': '정가람'}
{'흥행지수': 25773, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '배우이름': '진경'}
{'흥행지수': 19331, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '배우이름': '박지환'}
{'흥행지수': 12888, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '배우이름': '김준한'}
{'흥행지수': 6549, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '히트맨'], '배우이름': '허동원'}
{'흥행지수': 536, '출연작': ['불한당: 나쁜 놈들의 세상', '극한직업'], '배우이름': '진선규'}
{'흥행지수': 310, '출연작': ['시동', '극한직업'], '배우이름': '김종수'}


In [24]:
docs = actors_db.find({'$nor':[{'흥행지수':{'$in':[9182,8439]}},{'흥행지수':{'$gt':10000}}]},
                     {'배우이름':1,'흥행지수':1,'_id':0})
for doc in docs:
    print(doc)

{'흥행지수': 8527, '배우이름': '윤승아'}
{'흥행지수': 8472, '배우이름': '윤세아'}
{'흥행지수': 6549, '배우이름': '허동원'}
{'흥행지수': 4788, '배우이름': '지창욱'}
{'흥행지수': 4484, '배우이름': '송강호'}
{'흥행지수': 4268, '배우이름': '배유람'}
{'흥행지수': 4255, '배우이름': '김소현'}
{'흥행지수': 4236, '배우이름': '김용림'}
{'흥행지수': 3723, '배우이름': '이레'}
{'흥행지수': 3333, '배우이름': '하정우'}
{'흥행지수': 2855, '배우이름': '심은경'}
{'흥행지수': 2852, '배우이름': '장혜진'}
{'흥행지수': 2591, '배우이름': '김남길'}
{'흥행지수': 2568, '배우이름': '박소담'}
{'흥행지수': 2478, '배우이름': '이정은'}
{'흥행지수': 2478, '배우이름': '홍근택'}
{'흥행지수': 2349, '배우이름': '오지호'}
{'흥행지수': 2316, '배우이름': '이선균'}
{'흥행지수': 2223, '배우이름': '허율'}
{'흥행지수': 2091, '배우이름': '김상호'}
{'흥행지수': 1997, '배우이름': '조여정'}
{'흥행지수': 1984, '배우이름': '차지현'}
{'흥행지수': 1958, '배우이름': '차지현'}
{'흥행지수': 1939, '배우이름': '김시아'}
{'흥행지수': 1803, '배우이름': '양정화'}
{'흥행지수': 1713, '배우이름': '정지소'}
{'흥행지수': 1671, '배우이름': '임희철'}
{'흥행지수': 1568, '배우이름': '지성원'}
{'흥행지수': 1516, '배우이름': '장리우'}
{'흥행지수': 1487, '배우이름': '장준현'}
{'흥행지수': 1465, '배우이름': '한동규'}
{'흥행지수': 1461, '배우이름': '전혜진'}
{'흥행지수': 1449, '배우이름': '박재한'}
{'흥행지수': 142

In [26]:
docs = actors_db.find({'흥행지수':{'$gte':10000}}).skip(3)
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8869c8155a38268c005b67'), '배우순위': '4', '흥행지수': 57989, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '직업': '배우', '생년월일': '1972-11-21', '성별': '남', '신장/체중': '177cm, 72kg', '배우이름': '배성우'}
{'_id': ObjectId('5e8869c8155a38268c005b68'), '배우순위': '5', '흥행지수': 50847, '출연작': ['정직한 후보'], '직업': '배우', '생년월일': '1975-03-06', '성별': '여', '홈페이지': 'https://www.facebook.com/cjesmiran', '배우이름': '라미란'}
{'_id': ObjectId('5e8869c8155a38268c005b69'), '배우순위': '6', '흥행지수': 46607, '출연작': ['정직한 후보'], '직업': '배우', '생년월일': '1982-05-22', '성별': '남', '홈페이지': 'https://twitter.com/k_moo', '배우이름': '김무열'}
{'_id': ObjectId('5e8869c8155a38268c005b6a'), '배우순위': '7', '흥행지수': 45207, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '부당거래', '창궐'], '직업': '배우', '성별': '남', '배우이름': '정만식'}
{'_id': ObjectId('5e8869c8155a38268c005b6b'), '배우순위': '8', '흥행지수': 42372, '출연작': ['정직한 후보'], '직업': '배우', '생년월일': '1941-11-30', '성별': '여', '홈페이지': 'http://www.namuni.kr/', '학교': '창덕여자고등학교', '배우이름': '나문희'}
{'_id': ObjectId('5e8869c8155a38268c005b6c'), '배우순위': '9', '흥행지수'

In [27]:
docs = actors_db.find({'흥행지수':{'$gte':10000}}).limit(3)
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8869c8155a38268c005b64'), '배우순위': '1', '흥행지수': 70877, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '다른 이름': 'Jeon Do Youn', '직업': '배우', '생년월일': '1973-02-11', '성별': '여', '신장/체중': '165cm, 45kg', '학교': '서울예술대학 방송연예', '취미': '음악듣기, 영화감상', '특기': '수상스키, 포켓볼', '소속사': '노아엔터테인먼트', '배우이름': '전도연'}
{'_id': ObjectId('5e8869c8155a38268c005b65'), '배우순위': '2', '흥행지수': 68603, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '찬실이는 복도 많지'], '원어명': '윤여정', '직업': '배우', '생년월일': '1947-06-19', '성별': '여', '학교': '한양대학교 국문학 (중퇴)', '배우이름': '윤여정'}
{'_id': ObjectId('5e8869c8155a38268c005b66'), '배우순위': '3', '흥행지수': 64632, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '비트'], '직업': '배우', '생년월일': '1973-03-20', '성별': '남', '홈페이지': 'https://www.instagram.com/tojws/\nhttps://www.facebook.com/officialjws', '신장/체중': '186cm, 79kg', '학교': '경기상업고등학교 중퇴', '취미': '비디오 감상，드라이브', '특기': '농구，수영', '소속사': '(주)토러스 필름', '배우이름': '정우성'}


In [28]:
docs = actors_db.find({'흥행지수':{'$gte':10000}}).skip(3).limit(5)
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8869c8155a38268c005b67'), '배우순위': '4', '흥행지수': 57989, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '직업': '배우', '생년월일': '1972-11-21', '성별': '남', '신장/체중': '177cm, 72kg', '배우이름': '배성우'}
{'_id': ObjectId('5e8869c8155a38268c005b68'), '배우순위': '5', '흥행지수': 50847, '출연작': ['정직한 후보'], '직업': '배우', '생년월일': '1975-03-06', '성별': '여', '홈페이지': 'https://www.facebook.com/cjesmiran', '배우이름': '라미란'}
{'_id': ObjectId('5e8869c8155a38268c005b69'), '배우순위': '6', '흥행지수': 46607, '출연작': ['정직한 후보'], '직업': '배우', '생년월일': '1982-05-22', '성별': '남', '홈페이지': 'https://twitter.com/k_moo', '배우이름': '김무열'}
{'_id': ObjectId('5e8869c8155a38268c005b6a'), '배우순위': '7', '흥행지수': 45207, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '부당거래', '창궐'], '직업': '배우', '성별': '남', '배우이름': '정만식'}
{'_id': ObjectId('5e8869c8155a38268c005b6b'), '배우순위': '8', '흥행지수': 42372, '출연작': ['정직한 후보'], '직업': '배우', '생년월일': '1941-11-30', '성별': '여', '홈페이지': 'http://www.namuni.kr/', '학교': '창덕여자고등학교', '배우이름': '나문희'}


In [29]:
docs = actors_db.find({'출연작':'극한직업'})
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8869c8155a38268c005bce'), '배우순위': '107', '흥행지수': 536, '출연작': ['불한당: 나쁜 놈들의 세상', '극한직업'], '직업': '배우', '생년월일': '1977-09-13', '성별': '남', '배우이름': '진선규'}
{'_id': ObjectId('5e8869c8155a38268c005bec'), '배우순위': '137', '흥행지수': 310, '출연작': ['시동', '극한직업'], '직업': '배우', '생년월일': '1964-11-30', '성별': '남', '홈페이지': 'https://www.instagram.com/alrun85/\nhttps://www.facebook.com/profile.php?id=100001721292638', '배우이름': '김종수'}


In [32]:
docs = actors_db.find({'$or':[{'출연작':'지푸라기라도 잡고 싶은 짐승들'}, {'출연작':'정직한 후보'}]})
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8869c8155a38268c005b64'), '배우순위': '1', '흥행지수': 70877, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '다른 이름': 'Jeon Do Youn', '직업': '배우', '생년월일': '1973-02-11', '성별': '여', '신장/체중': '165cm, 45kg', '학교': '서울예술대학 방송연예', '취미': '음악듣기, 영화감상', '특기': '수상스키, 포켓볼', '소속사': '노아엔터테인먼트', '배우이름': '전도연'}
{'_id': ObjectId('5e8869c8155a38268c005b65'), '배우순위': '2', '흥행지수': 68603, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '찬실이는 복도 많지'], '원어명': '윤여정', '직업': '배우', '생년월일': '1947-06-19', '성별': '여', '학교': '한양대학교 국문학 (중퇴)', '배우이름': '윤여정'}
{'_id': ObjectId('5e8869c8155a38268c005b66'), '배우순위': '3', '흥행지수': 64632, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '비트'], '직업': '배우', '생년월일': '1973-03-20', '성별': '남', '홈페이지': 'https://www.instagram.com/tojws/\nhttps://www.facebook.com/officialjws', '신장/체중': '186cm, 79kg', '학교': '경기상업고등학교 중퇴', '취미': '비디오 감상，드라이브', '특기': '농구，수영', '소속사': '(주)토러스 필름', '배우이름': '정우성'}
{'_id': ObjectId('5e8869c8155a38268c005b67'), '배우순위': '4', '흥행지수': 57989, '출연작': ['지푸라기라도 잡고 싶은 짐승들'], '직업': '배우', '생년월일': '1972-11-21', '성별':

In [35]:
docs = actors_db.find({'출연작':{'$all':['백두산','정직한 후보']}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8869c8155a38268c005b74'), '배우순위': '17', '흥행지수': 21328, '출연작': ['정직한 후보', '백두산'], '직업': '배우', '생년월일': '1973-05-13', '성별': '남', '배우이름': '조한철'}


In [36]:
docs = actors_db.find({'출연작.0':'정직한 후보'})
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8869c8155a38268c005b68'), '배우순위': '5', '흥행지수': 50847, '출연작': ['정직한 후보'], '직업': '배우', '생년월일': '1975-03-06', '성별': '여', '홈페이지': 'https://www.facebook.com/cjesmiran', '배우이름': '라미란'}
{'_id': ObjectId('5e8869c8155a38268c005b69'), '배우순위': '6', '흥행지수': 46607, '출연작': ['정직한 후보'], '직업': '배우', '생년월일': '1982-05-22', '성별': '남', '홈페이지': 'https://twitter.com/k_moo', '배우이름': '김무열'}
{'_id': ObjectId('5e8869c8155a38268c005b6b'), '배우순위': '8', '흥행지수': 42372, '출연작': ['정직한 후보'], '직업': '배우', '생년월일': '1941-11-30', '성별': '여', '홈페이지': 'http://www.namuni.kr/', '학교': '창덕여자고등학교', '배우이름': '나문희'}
{'_id': ObjectId('5e8869c8155a38268c005b6d'), '배우순위': '10', '흥행지수': 38349, '출연작': ['정직한 후보', '시동'], '직업': '배우', '생년월일': '1980-07-05', '성별': '남', '배우이름': '윤경호'}
{'_id': ObjectId('5e8869c8155a38268c005b6e'), '배우순위': '11', '흥행지수': 33897, '출연작': ['정직한 후보'], '직업': '배우', '성별': '남', '홈페이지': 'http://www.vlive.tv/channels/D5353F', '배우이름': '장동주'}
{'_id': ObjectId('5e8869c8155a38268c005b70'), '배우순위': '13', '흥행지수': 

In [37]:
docs = actors_db.find({'출연작':{'$size':3}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5e8869c8155a38268c005b6a'), '배우순위': '7', '흥행지수': 45207, '출연작': ['지푸라기라도 잡고 싶은 짐승들', '부당거래', '창궐'], '직업': '배우', '성별': '남', '배우이름': '정만식'}
{'_id': ObjectId('5e8869c8155a38268c005b8a'), '배우순위': '39', '흥행지수': 2478, '출연작': ['용길이네 곱창집', '기생충', '미스터 주: 사라진 VIP'], '직업': '배우', '생년월일': '1970-01-23', '성별': '여', '학교': '한양대학교', '배우이름': '이정은'}
{'_id': ObjectId('5e8869c8155a38268c005bd5'), '배우순위': '114', '흥행지수': 490, '출연작': ['눈의 여왕4', '극장판 미니특공대: 공룡왕 디노', '극장판 원피스 스탬피드'], '직업': '성우', '성별': '남', '배우이름': '남도형'}
{'_id': ObjectId('5e8869c8155a38268c005bd6'), '배우순위': '115', '흥행지수': 477, '출연작': ['남산의 부장들', '부당거래', '미스터 주: 사라진 VIP'], '직업': '배우', '생년월일': '1968-10-15', '성별': '남', '신장/체중': '178cm', '배우이름': '이성민'}


In [38]:
conn = pymongo.MongoClient()
cine21 = conn.cine21
elematch_sample = cine21.sample

In [39]:
elematch_sample.insert_many([
    {'result':[88,75,91]},
    {'result':[82,85,89]}
])

In [42]:
docs = elematch_sample.find({'result':{'$elemMatch':{'$gte':88, '$lte':89}}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5e89b88540acd094bec5c26b'), 'result': [88, 75, 91]}
{'_id': ObjectId('5e89b88540acd094bec5c26c'), 'result': [82, 85, 89]}
